In [1]:
import os
import numpy as np
import cv2
import keras
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model, Sequential
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [2]:
#Loda image
path_daisy = 'C:/Users/AlbertHsu/Documents/GitHub/4th-ML100Days/homework/Data/image_data/train/daisy/'
path_dandelion = 'C:/Users/AlbertHsu/Documents/GitHub/4th-ML100Days/homework/Data/image_data/train/dandelion/'
path_rose = 'C:/Users/AlbertHsu/Documents/GitHub/4th-ML100Days/homework/Data/image_data/train/rose/'
path_sunflower = 'C:/Users/AlbertHsu/Documents/GitHub/4th-ML100Days/homework/Data/image_data/train/sunflower/'
path_tulip = 'C:/Users/AlbertHsu/Documents/GitHub/4th-ML100Days/homework/Data/image_data/train/tulip/'


daisy = os.listdir(path_daisy)
dandelion = os.listdir(path_dandelion)
rose = os.listdir(path_rose)
sunflower = os.listdir(path_sunflower)
tulip = os.listdir(path_tulip)

In [3]:
#crate Lable array y 
y_daisy = [0]*len(daisy)
y_dandelion = [1]*len(dandelion)
y_rose = [2]*len(rose)
y_sunflower = [3]*len(sunflower)
y_tulip = [4]*len(tulip)

In [4]:
#Load image to arrays
img_flowers = []
img_dandelion = np.zeros((len(dandelion),1))
img_rose = np.zeros((len(rose),1))
img_sunflower = np.zeros((len(sunflower),1))
img_tulip = np.zeros((len(tulip),1))

Downsize = 256

i = 0
for i in range(len(daisy)):
    pic = cv2.imread(path_daisy+daisy[i])
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(Downsize,Downsize))
    img_flowers.append(pic)
    
i = 0
for i in range(len(dandelion)):
    pic = cv2.imread(path_dandelion+dandelion[i])
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(Downsize,Downsize))
    img_flowers.append(pic)
    
i = 0
for i in range(len(rose)):
    pic = cv2.imread(path_rose+rose[i])
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(Downsize,Downsize))
    img_flowers.append(pic)
    
i = 0
for i in range(len(sunflower)):
    pic = cv2.imread(path_sunflower+sunflower[i])
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(Downsize,Downsize))
    img_flowers.append(pic)
    
i = 0
for i in range(len(tulip)):
    pic = cv2.imread(path_tulip+tulip[i])
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(Downsize,Downsize))
    img_flowers.append(pic)
    

    
print(len(img_flowers))

2823


In [5]:
# Combine Data(img_flowers) and Label(y___)

Y = y_daisy+y_dandelion+y_rose+y_sunflower+y_tulip
Y = np.array(Y)
Y = np.reshape(Y,(Y.shape[0],1))

dataAndLabel = []
for i in range(Y.shape[0]):
    dataAndLabel.append([img_flowers[i],Y[i,0]])
    
dataAndLabel = np.array(dataAndLabel)

##打亂資料排列
np.random.seed(110)
np.random.shuffle(dataAndLabel)

In [6]:
# Reorganize data and label to X(data) Y(Label). Then Split to X_test X_train, Y_test Y_train
# And normalization
X_data = []
Y_data = []
for i in range(Y.shape[0]):
    X_data.append(dataAndLabel[i][0])
    Y_data.append(dataAndLabel[i][1])

X_data = np.array(X_data)
Y_data = np.array(Y_data)
Y_data = np.reshape(Y_data,(Y_data.shape[0],1))
Y_data = to_categorical(Y_data)

#normalization
X_data =  X_data/255 

#Train Test Split
x_train,x_test,y_train,y_test = train_test_split(X_data,Y_data,test_size=0.20,random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(2258, 256, 256, 3)
(2258, 5)
(565, 256, 256, 3)
(565, 5)


In [7]:
# 設定循環類別參數
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 5 # 類別的數量，共有 5 個類別
epochs = 70 # 訓練整個資料集共 30個循環
IMAGE_SIZE0 = Downsize
IMAGE_SIZE1 = Downsize

In [9]:
# Case1:建立神經網路 這邊直接用ResNet50. 如果要接續訓練，用

resnetModel = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE0,IMAGE_SIZE1,3))
x = resnetModel.output
x = Flatten()(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
x = BatchNormalization()(x)
#x = Dense(2048, activation='relu', name = "outputL1")(x)
#x = BatchNormalization()(x)
x = Dense(1024, activation='relu', name = "outputL2")(x)
x = BatchNormalization()(x)

output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

resnetModel50 = Model(inputs=resnetModel.input, outputs=output_layer)

resnetModel50.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [8]:
# Case2: load我之前存好的model繼續訓練 不要跟Case1同時跑
resnetModel50 = keras.models.load_model("./ML100_Final_Flower.h5")

In [9]:
# 載入 Callbacks
from keras.callbacks import ModelCheckpoint

model_ckpt = ModelCheckpoint(filepath="./ML100_Final_Flower.h5", 
                             monitor="val_loss", 
                             save_best_only=True)


In [ ]:
# Train Model using image augment generator
#epochs = 70

augment_generator = ImageDataGenerator(rotation_range=40, width_shift_range=0.4, height_shift_range=0.4, shear_range=0.2,
    zoom_range=0.3,horizontal_flip=True)

history = resnetModel50.fit_generator(augment_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[model_ckpt])

#score = model.evaluate(x_test, y_test, verbose=0)
score = resnetModel50.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/70
70/70 [==============================] - 3997s 57s/step - loss: 0.6958 - accuracy: 0.7695 - val_loss: 0.8747 - val_accuracy: 0.7009
Epoch 2/70
70/70 [==============================] - 3944s 56s/step - loss: 0.7451 - accuracy: 0.7381 - val_loss: 0.8925 - val_accuracy: 0.6973
Epoch 3/70
70/70 [==============================] - 4038s 58s/step - loss: 0.7111 - accuracy: 0.7682 - val_loss: 0.6611 - val_accuracy: 0.7752
Epoch 4/70
70/70 [==============================] - 4089s 58s/step - loss: 0.6622 - accuracy: 0.7794 - val_loss: 0.7781 - val_accuracy: 0.7504
Epoch 5/70
70/70 [==============================] - 4244s 61s/step - loss: 0.6344 - accuracy: 0.7808 - val_loss: 0.6836 - val_accuracy: 0.7646
Epoch 6/70
70/70 [==============================] - 4256s 61s/step - loss: 0.6550 - accuracy: 0.7803 - val_loss: 0.7611 - val_accuracy: 0.7381
Epoch 7/70
70/70 [==============================] - 4291s 61s/step - loss: 0.6942 - accuracy: 0.7520 - val_loss: 91.4867 - val_accuracy: 0.417

In [8]:
#Save latest model for furthr trainning
#resnetModel50.save("ML100_Final_Flower_latest.h5")

#Or Load Model to test
#resnetModel50 = keras.models.load_model("./ML100_Final_Flower_latest.h5") #latest model
resnetModel50 = keras.models.load_model("./ML100_Final_Flower.h5") # Best model

In [9]:
#Load test Data, Normalization and predict
#TestModel = keras.models.load_model("./ML100_Final_Flower.h5")
path_testImg = 'C:/Users/AlbertHsu/Documents/GitHub/4th-ML100Days/homework/Data/image_data/test/'
testImg = os.listdir(path_testImg)
#img_test = np.zeros((len(testImg),1))
img_test = []
Downsize = 256

i = 0
for i in range(len(testImg)):
    pic = cv2.imread(path_testImg+testImg[i])
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(Downsize,Downsize))
    img_test.append(pic)

img_test = np.array(img_test)
img_test = img_test/255

pred = resnetModel50.predict(img_test)

In [21]:
print(pred)

[[1.7468415e-02 6.0330154e-03 8.7225425e-01 8.4274355e-03 9.5816866e-02]
 [1.8978843e-02 4.0312104e-02 6.2691256e-02 2.4774894e-02 8.5324287e-01]
 [1.6956350e-03 6.5342046e-04 9.1229945e-01 4.3783397e-03 8.0973104e-02]
 ...
 [9.9642605e-01 1.5503658e-05 2.8585012e-03 2.3699409e-04 4.6291997e-04]
 [4.2249519e-02 9.2686899e-02 4.1679449e-02 6.7655069e-01 1.4683346e-01]
 [8.6006457e-03 2.3005197e-03 4.5166206e-01 9.1225776e-04 5.3652453e-01]]


In [10]:
pred = np.array(pred)
pred_forsave = np.argmax(pred, axis=1)
#print(pred_forsave)
#print(testImg)
testImgId = []
for x in testImg:
    testImgId.append(os.path.splitext(x)[0])

In [11]:
# Save to CSV file for submission
subcsv = pd.DataFrame({'id': testImgId, 'flower_class': pred_forsave})
subcsv.to_csv('ml100FinalSub1.csv', index=False)